In [1]:
%pip install mmh3 numpy

## The Bloom embeddings algorithm

In a normal embedding table, each word-string is mapped to a distinct ID.
Usually these IDs will be sequential, so if you have a vocabulary of 100 words,
your words will be mapped to numbers `range(100)`. The sequential IDs can then
adrianeboyd marked this conversation as resolved.
be used as indices into an embedding table: if you have 100 words in your
adrianeboyd marked this conversation as resolved.
vocabulary, you have 100 rows in the table, and each word receives its own
vector.

However, there's no limit to the number of unique words that might occur in a
sample of text, while we definitely want a limited number of rows in our
embedding table. Some of the rows in our table will therefore need to be shared
between multiple words in our vocabulary. One obvious solution is to set aside a
single vector in the table. Words 0-98 will each receive their own vector, while
all other words are assigned to vector 99.

However, this asks vector 99 to do a lot of work. What if we gave more vectors
to the unknown words?

In [2]:
def get_row(word_id, number_vector=100, number_oov=10):
    if word_id < (number_vector - number_oov):
        return word_id
    else:
        return number_vector + (word_id % number_oov)

This gives the model a little more resolution for the unknown words. If all
out-of-vocabulary words are assigned the same vector, then they'll all look
identical to the model. Even if the training data actually includes information
that shows two different out-of-vocabulary words have important, different
implications -- for instance, if one word is a strong indicator of positive
sentiment, while the other is a strong indicator of negative sentiment -- the
model won't be able to tell them apart. However, if we have 10 buckets for the
unknown words, we might get lucky, and assign these words to different buckets.
If so, the model would be able to learn that one of the unknown-word vectors
makes positive sentiment more likely, while the other vector makes negative
sentiment more likely.

If this is good, then why not do more of it? Bloom embeddings are like an
extreme version, where _every_ word is handled like the unknown words above:
there are 100 vectors for the "unknown" portion, and 0 for the "known" portion.

So far, this approach seems weird, but not necessarily good. The part that makes
it unfairly effective is the next step: by simply doing the same thing multiple
times, we can greatly improve the resolution, and have unique representations
for far more words than we have vectors. The code in full:

In [3]:
import numpy
import mmh3

def allocate(n_vectors, n_dimensions):
    table = numpy.zeros((n_vectors, n_dimensions), dtype='f')
    table += numpy.random.uniform(-0.1, 0.1, table.size).reshape(table.shape)
    return table

def get_vector(table, word):
    hash1 = mmh3.hash(word, seed=0)
    hash2 = mmh3.hash(word, seed=1)
    row1 = hash1 % table.shape[0]
    row2 = hash2 % table.shape[0]
    return table[row1] + table[row2]

def update_vector(table, word, d_vector):
    hash1 = mmh3.hash(word, seed=0)
    hash2 = mmh3.hash(word, seed=1)
    row1 = hash1 % table.shape[0]
    row2 = hash2 % table.shape[0]
    table[row1] -= 0.001 * d_vector
    table[row2] -= 0.001 * d_vector

In this example, we've used two keys, assigned from two random hash functions.
It's unlikely that two words will collide on both keys, so by simply summing the
vectors together, we'll assign most words a unique representation.

For the sake of illustration, let's step through a very small example,
explicitly.

Let's say we have this vocabulary of 20 words:

In [4]:
vocab = ['apple', 'strawberry', 'orange', 'juice', 'drink', 'smoothie',
         'eat', 'fruit', 'health', 'wellness', 'steak', 'fries', 'ketchup',
         'burger', 'chips', 'lobster', 'caviar', 'service', 'waiter', 'chef']

We'll embed these into two dimensions. Normally this would give us a table of
`(20, 2)` floats, which we would randomly initialise. With the hashing trick, we
can make the table smaller. Let's give it 15 vectors:

In [5]:
normal_embed = numpy.random.uniform(-0.1, 0.1, (20, 2))
hashed_embed = numpy.random.uniform(-0.1, 0.1, (15, 2))

In the normal table, we want to map each word in our vocabulary to its own
vector:

In [6]:
word2id = {}
def get_normal_vector(word, table):
    if word not in word2id.keys():
        word2id[word] = len(word2id)
    return table[word2id[word]]

The hashed table only has 15 rows, so some words will have to share. We'll
handle this by mapping the word into an arbitrary integer – called a "hash
value". The hash function will return an arbitrary integer, which we'll mod into
the range `(0, 15)`. Importantly, we need to be able to compute _multiple,
distinct_ hash values for each key – so Python's built-in hash function is
inconvenient. We'll therefore use MurmurHash.

Let's see what keys we get for our 20 vocabulary items, using MurmurHash:

In [7]:
hashes1 = [mmh3.hash(w, 1) % 15 for w in vocab]
assert hashes1 == [3, 6, 4, 13, 8, 3, 13, 1, 9, 12, 11, 4, 2, 13, 5, 10, 0, 2, 10, 13]

As you can see, some keys are shared between multiple words, while 2/15 keys are
unoccupied. This is obviously unideal! If multiple words have the same key,
they'll map to the same vector – as far as the model is concerned, "strawberry"
and "heart" will be indistinguishable. It won't be clear which word was used –
they have the same representation.

To address this, we simply hash the words again, this time using a different
seed – so that we get a different set of arbitrary keys:

In [8]:
from collections import Counter

hashes2 = [mmh3.hash(w, 2) % 15 for w in vocab]
assert len(Counter(hashes2).most_common()) == 12

This one's even worse – 3 keys unoccupied! But our strategy is not to keep drawing until we get a favorable seed. Instead, consider this:

In [9]:
assert len(Counter(zip(hashes1, hashes2))) == 20

By combining the results from the two hashes, our 20 words distribute perfectly,
into 20 unique combinations. This makes sense: we expect to have some words
overlapping on one of the keys, but we'd have to be very unlucky for a pair of
words to overlap on _both_ keys.

This means that if we simply add the two vectors together, each word once more
has a unique representation:

In [10]:
for word in vocab:
    key1 = mmh3.hash(word, 0) % 15
    key2 = mmh3.hash(word, 1) % 15
    vector = hashed_embed[key1] + hashed_embed[key2]
    print(word, '%.3f %.3f' % tuple(vector))

apple -0.033 -0.012
strawberry -0.023 -0.037
orange 0.158 -0.031
juice -0.045 0.139
drink 0.024 0.030
smoothie 0.121 0.076
eat -0.093 0.153
fruit 0.083 0.052
health 0.064 -0.046
wellness 0.143 0.112
steak 0.011 -0.097
fries 0.036 0.041
ketchup 0.081 0.029
burger -0.045 0.139
chips -0.118 -0.090
lobster 0.016 -0.107
caviar -0.033 -0.012
service 0.081 0.029
waiter 0.179 -0.038
chef -0.047 0.062


We now have a function that maps our 20 words to 20 unique vectors – but we're
storing weights for only 15 vectors in memory. Now the question is: will we be
able to find values for these weights that let us actually map words to useful
vectors?

Let's do a quick experiment to see how this works. We'll assign "true" values
for our little vocabulary, and see how well we can approximate them with our
compressed table. To get the "true" values, we _could_ put the "science" in data
science, and drag the words around into reasonable-looking clusters. But for our
purposes, the actual "true" values don't matter. We'll therefore just do a
simulation: we'll assign random vectors as the "true" state, and see if we can
learn values for the hash embeddings that match them.

The learning procedure will be a simple stochastic gradient descent:

In [11]:
import numpy
import numpy.random as random
import mmh3

random.seed(0)
nb_epoch = 500
learn_rate = 0.001
nr_hash_vector = 1000

words = [str(i) for i in range(2000)]
true_vectors = numpy.random.uniform(-0.1, 0.1, (len(words), 10))
hash_vectors = numpy.random.uniform(-0.1, 0.1, (nr_hash_vector, 10))
examples = list(zip(words, true_vectors))

for epoch in range(nb_epoch):
    random.shuffle(examples)
    loss=0.
    for word, truth in examples:
        key1 = mmh3.hash(word, 0) % nr_hash_vector
        key2 = mmh3.hash(word, 1) % nr_hash_vector
        hash_vector = hash_vectors[key1] + hash_vectors[key2]
        diff = hash_vector - truth
        hash_vectors[key1] -= learn_rate * diff
        hash_vectors[key2] -= learn_rate * diff
        loss += (diff**2).sum()
print(epoch, loss)

499 43.47128495286


It's worth taking some time to play with this simulation. You can start by doing
some sanity checks:

- How does the loss change with `nr_hash_vector`?
- If you remove `key2`, does the loss go up?
- What happens if you add more hash keys?
- What happens as the vocabulary size increases?
- What happens when more dimensions are added?
- How sensitive are the hash embeddings to the initial conditions? If we change the random seed, do we ever get unlucky?

If you play with the simulation for a while, you'll start to get a good feel for
the dynamics, and hopefully you'll have a clear idea of why the technique works.

## Bonus Section 

To make it easier for folks to try out a whole bunch of settings we'd added a little bit of code below that makes it easier to get relevant visuals.

In [16]:
%pip install altair pandas

In [14]:
from functools import reduce 


def calc_losses(epochs=500, seed=0, learn_rate=0.001, nr_hash_vector=1000, n_hash=3, n_words=1000, size_vector=10):
    random.seed(seed)
    nb_epoch = epochs
    learn_rate = learn_rate
    nr_hash_vector = nr_hash_vector

    words = [str(i) for i in range(n_words)]
    true_vectors = numpy.random.uniform(-0.1, 0.1, (len(words), size_vector))
    hash_vectors = numpy.random.uniform(-0.1, 0.1, (nr_hash_vector, size_vector))
    examples = list(zip(words, true_vectors))

    losses = []
    for epoch in range(nb_epoch):
        random.shuffle(examples)
        loss=0.
        for word, truth in examples:
            keys = [mmh3.hash(word, k) % nr_hash_vector for k in range(n_hash)]
            hash_vector = reduce(lambda a, b: a + b, [hash_vectors[k] for k in keys])
            diff = hash_vector - truth
            for key in keys:
                hash_vectors[key] -= learn_rate * diff
            loss += (diff**2).sum()
        losses.append(loss)
    return losses

data = []
for n_hash in [1, 2, 3, 4, 5]:
    losses = calc_losses(nr_hash_vector=2_000, n_words=10_000, n_hash=n_hash, epochs=150)
    data = data + [{"loss": l, "nr_hash_vector": nr_hash_vector, "n_hash": str(n_hash), "epoch": e} for e, l in enumerate(losses)]

In [15]:
import pandas as pd
import altair as alt

source = pd.DataFrame(data)

(alt.Chart(source)
  .mark_line()
  .encode(x='epoch', y='loss', color='n_hash')
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)